# TAA4: Accessibility to healthcare facilities

In this tutorial, we wrap up what you have learned in TAA1-3 by demonstrating their applications and connections. For a European country of your choice, we collect data on the population and their health facilities. Using a classification method, we add rural and urban features to the population data points. Then, with a clustering algorithm, we group population points based on their coordinates and assign each cluster to its closest hospitals. This allows us to calculate the total urban and rural demand for each hospital. Next, we assess the impact of flooding on the hospitals, distinguishing between damaged and undamaged facilities.

In the aftermath of the flood, the number of hospitals in service has changed. Your task will be to repeat the clustering process, assign the new clusters to the intact hospitals, and then calculate their urban and rural demand in the post-flood conditions.

### Important before we start
---
Make sure that you save this file before you continue, else you will lose everything. To do so, go to **Bestand/File** and click on **Een kopie opslaan in Drive/Save a Copy on Drive**!

## Learning Objectives

- To extract, prepare and manipulate geospatial information

- To run a classification algorithm to identify urban and rural land use.

- To overlay raster and vector information.

- To cluster different geospatial layers.

- To visualise geospatial information.
<hr>



## Prepare the packages
<hr>

Before we can start, we need to install several packages. Most of you, you have seen before in the previous tutorials!

In [ ]:
!pip install -q rasterio rioxarray contextily osm_flex exactextract

Now we will import these packages in the cell below:

In [ ]:
# Standard Library Imports
import os
import sys
from pathlib import Path
from datetime import datetime
from zipfile import ZipFile
from io import BytesIO
import random
import requests
from urllib.request import urlopen

# Data Manipulation and Analysis
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray as rxr

# Machine Learning
import sklearn  # General import if other sklearn modules are needed
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans

# Geometry and Spatial Analysis
import shapely
from shapely.geometry import Point
import rasterio as rio
from rasterio.enums import Resampling
from scipy.spatial.distance import cdist

# Earth Engine and Geospatial Libraries
import ee
import geemap
import contextily as cx
import osm_flex
from osm_flex import download
from exactextract import exact_extract

# Visualization
import matplotlib.pyplot as plt
import contextily as ctx
from tqdm import tqdm
from IPython.display import clear_output

## 2. Data download and preparation

Define a country of your interest and a size for gridding and a randomSeed.

We use the ISO3 code of the country to select it from the data. Please have a look online yourself if you do not know the ISO3 code of your country (it is widely used within geospatial analysis).

In [ ]:
country_full_name = 'XXX' # add full name of the country
country_iso3 = 'XXX' # add ISO3 code of the country.
upscale_factor = 100 # our population data that we will upscale to a lower resolution in a
                      #later stage, will have to be converted from 100x100m to something more useful (such as 10x10km)

### Set global seeds ###
random_seed = 1
np.random.seed(random_seed)
random.seed(random_seed)
os.environ['PYTHONHASHSEED'] = str(random_seed)

Here, we download the population data from WorldPop, an open source platform. Select the country of interest from the WorldPop [website](https://hub.worldpop.org/geodata/listing?id=69) and add the link to the URL below.

In [ ]:
url = "XXXX"

file_name = 'ppp_2018_1km_Aggregated.tif'

open(file_name, 'wb').write(requests.get(url).content)

world_pop = xr.open_dataset(file_name,engine='rasterio')

Now, we use a file with country borders from Natural Earth, to get the boundaries of the country of your interest.

In [ ]:
world = gpd.read_file("https://github.com/nvkelso/natural-earth-vector/raw/master/10m_cultural/ne_10m_admin_0_countries.shp")

# And we want to take the country boundaries and geometry
country_bounds = world.loc[world.ADM0_ISO == country_iso3].bounds
country_geom = world.loc[world.ADM0_ISO == country_iso3].geometry

Check if the country_bounds and country_geom are not empty. If not empty, you selected an existing ISO3 code!

Now, we use the derived boundries to clip the population data from worldpop, to get the population of our country. We use the **rio.clip_box(** and **rio.cip(** function again. We used those before in TAA4. Apply them here to clip the world_pop data, to make sure it for sure fits the same spatial extent as the hospital data.

In [ ]:
world_pop_national = world_pop.rio.clip_box(minx..

                    )
world_pop_national = world_pop_national.rio.clip(XX, XX, drop=False)

The worldpop data is stored as 100 by 100m grid. This will be too computationally intensive if we would use that resolution. As such, we reproject the to a lower resolution. This will help us to perform the analyis more smoothly. We use the *upscale_factor* as defined at the start of this subsection.

In [ ]:
new_width = int(world_pop_national.rio.width / upscale_factor)
new_height = int(world_pop_national.rio.height / upscale_factor)

worldpop_Grided = world_pop_national.rio.reproject(
    world_pop_national.rio.crs,
    shape=(new_height, new_width),
    resampling=Resampling.sum,
)

Now we remove the missing data from our data points and create a GeoDataFrame for our country.

In [ ]:
df_worldpop_ = worldpop_Grided.band_data.to_dataframe()
df_worldpop_ = df_worldpop_.loc[~df_worldpop_.band_data.isna()].reset_index(drop=False)

# create geometry values and drop lat lon columns
df_worldpop_['geometry'] = shapely.points(np.array(list(zip(df_worldpop_['x'],df_worldpop_['y']))))

df_worldpop_ = gpd.GeoDataFrame(df_worldpop_.drop(['y','x','spatial_ref','band'],axis=1))

# dynamically create a variable name for the DataFrame
globals()[f'df_pop_{country_iso3}'] = gpd.GeoDataFrame(df_worldpop_) 

# dynamically create a print statement that reflects the current country code (however df_worldpop_ would work as well)
print(f"The output is df_pop_{country_iso3} as a dataframe of the population data of {country_full_name}")

And Lets plot the population points of our country of interest.

**Q: Make sure you provide a plot with all its characteristics (title, good colors etc).**

Our next step is to extract information of healthcare facilities for the country of interest. We do so using OpenStreetMap. With the latest version of geopandas, it is now possible to directly read **osm.pbf** files from OpenStreetMap.

In [ ]:
%%time
Country_GeofabrikData_path = download.get_country_geofabrik(country_iso3)

Healthcare facilities are stored as *multipolygons* within OpenStreetMap, and we want to download all clinics and hospitals. Depending on the country, this may take a while.

In [ ]:
HealthCenters = gpd.read_file(Country_GeofabrikData_path, layer="XXX")

sub_types =['XX', 'XX']
HealthCenters = HealthCenters[HealthCenters['amenity'].isin(sub_types)].reset_index(drop=True)

# to convert polygons to their centroids
HealthCenters_centroids = HealthCenters.copy()
HealthCenters_centroids['geometry'] = HealthCenters.centroid

Let's check the content of our generated HealthCenters_centroids GeoDataFrame.

In [ ]:
HealthCenters.head()

And let's visualize the hospitals locations.

If you want to use the contextily basemaps, you need to temporarily reproject the data to EPSG:3857 to add the basemap.


In [ ]:
print("The output is HealthCenters_centroids as a dataframe of the Health Centers")

#plotting
fig, ax = plt.subplots(figsize=(10, 10))
HealthCenters_centroids.plot(ax=ax, color='red', markersize=10, alpha=0.7)

#cx.add_basemap(ax, crs='EPSG:4326', source=cx.providers.OpenStreetMap.Mapnik, zoom=8)

ax.set_title()

ax.set_xlabel()
ax.set_ylabel()

plt.show()

## 3. Classification of rural and urban areas

As you remember, we checked the content of the population data, and there was no information regarding urban or rural areas. Therefore, we need to add this information to our dataset. So, we will use an approach similar to what you learned in TAA1, but for the sake of processing efficiency, we will use Earth Engine's built-in classifier packages rather than scikit-learn. In this situation we are interested in only two land cover classes: rural and urban. We will have to reclassify the raster, train a model to distinguish between these timesteps, and finally perform the classification on recent images.
<!--  -->
 We have pre-filled most of the code in this section, and you will use a few new tricks to perform the classification. Beyond that, we challenge you to leverage what you have learned in TAA1 to improve the performance of the model, by any means you see fit. You will be evaluated on your reasoning, as well as the creativity of the approaches.

### Set-up

First, set up your Earth Engine environment as you did before.

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project="XXX")

### Set-up training & validation data

We will make use of the following data sources:
1. Administrative country boundary to clip to the area of interest  
2. Sentinel-2 satellite images at 10x10m resolution
3. CORINE Land Cover (CLC)

For the satellite image, we take the S2 median image over the summer to improve sensitivity to single-capture conditions. The median image is taken over the summer, so that we exclude seasonal dynamics. Finally, we clip this image to the country extent.



In [ ]:
# Define the region of interest (ROI)
country_ROI = ee.FeatureCollection("FAO/GAUL/2015/level0") \
                .filter(ee.Filter.eq('ADM0_NAME', country_full_name))

# Load Sentinel-2 Image Collection
sentinel2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
             .filterBounds(country_ROI.geometry()) \
             .filterDate('2018-06-01', '2018-08-31') \
             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Create a median composite to reduce cloud cover
mosaic_image = sentinel2.median().clip(country_ROI.geometry())

Show on map to verify that it's loaded

In [ ]:
bands = [] # Fill in this list yourself
vis_params = {'max': , 'bands': bands}  # Limit upper range so you can see detail

map = geemap.Map(height=800, width=700, zoom=7)

# dynamically center the map on the selected country (ROI)
map.centerObject(country_ROI.geometry(), 8)

# adding the mosaic image layer for the selected country
map.addLayer(mosaic_image, vis_params, "Sentinel-2_2018")

map

### Calculate image variables to include

Next, we will compute variables to use for our classifier. We have provided the examples you've already seen for TAA1, but we challenge you to add your own variables. You can get inspiration from anywhere, but be sure that you are able to explain the reasoning behind including each variable.

In [ ]:
def make_s2_variables(s2_image):
    # Calculate additional spectral indices
    dvi = s2_image.select('B5').subtract(s2_image.select('B4')).rename('DVI')
    ndvi = s2_image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    ndwi = s2_image.normalizedDifference(['B3', 'B5']).rename('NDWI')

    # Add indices to the image
    s2_image = s2_image.addBands([dvi, ndvi, ndwi])

    '''
       ToDo: Add your own variables!

       Think about what you learned in TAA1 - how else can you compute information
       from the spectral bands to include? Look at some papers for inspiration,
       or try something out yourself. There's more than just indices that can help here!

       Don't forget to add them to the image after you create them!
    '''

    # Define neighborhood size (e.g., 3x3)

    # Calculate neighborhood statistics


        # Add additional statistics here, such as min, max, etc., if desired.

        # Append neighborhood bands to the list

    # Add neighborhood statistics to the image


    return s2_image

In [ ]:
mosaic_image = make_s2_variables(mosaic_image)

In [ ]:
# Use these bands for prediction
# DON'T FORGET to add your own variables when you make them!
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']
indices = ['NDVI', 'DVI']
img_bands = [*bands, *indices]

### Load CLC2018 and sample data


Now, let's load labels from CORINE and classify the model.

In [ ]:
CLC = ee.Image('COPERNICUS/CORINE/V20/100m/2018').select('landcover').clip(mosaic_image.geometry())
lc_points = CLC.sample(
    **{
        'region': mosaic_image.geometry(),
        'scale': 30,
        'numPixels': 10000, # Change this as you see fit
        'seed': random_seed,
        'geometries': True,
    }
)

Next, let's reclassify the dataset to a binary urban/rural dataset. The choice how to do this is yours, and can be as easy or complicated as you like it to be. As a reminder, CLC consists of a [3-digit hierarchy](https://land.copernicus.eu/content/corine-land-cover-nomenclature-guidelines/html/), so you can pick which classes you want to include and exclude.

In [ ]:
def generalize_clc_class(feature):
    lc_value = ee.String(feature.get('landcover'))

    # Check if the first character is '1'
    set_value = ee.Algorithms.If(lc_value.slice(0, 1).equals('1'), 1, 0)

    # Set the new binary value for the 'landcover' property
    return feature.set('landcover', set_value)

lc_reference_pts = lc_points.map(generalize_clc_class)

**Q: Briefly explain which classes you included for the urban/rural re-classification.**

Let's make training/validation splits. You can customize this however you see fit, but we suggest balanced sampling, where you control how many positive and how many negative samples are included during training/validation, so as to not over/under-sample one or the other.

In [ ]:
# Define the land cover labels column
label_col = 'landcover'

# Filter points by label

# Allow a maximum of 2-to-1 difference in negative vs positive class sampling

# Merge the samples


# Split into training and validation sets
training_sample = balanced_sample.filter('random <= 0.8')
validation_sample = balanced_sample.filter('random > 0.8')

# Sample regions for training and validation datasets
train_data = mosaic_image.select(img_bands).sampleRegions(
    collection=training_sample, properties=[label_col], scale=100
)

val_data = mosaic_image.select(img_bands).sampleRegions(
    collection=validation_sample, properties=[label_col], scale=100
)

**Q: Describe your sampling approach. Which approach did you go with, and what is the rationale behind it?**

### Optimize on training & validation set

Now that we have sampled data, we will iteratively improve the model. How you want to approach this is up to you, we only provide the basic process here. In TAA1 we taught you a few methods and concepts that you can build on. To give some suggestions:
1. Analyze the confusion matrix
2. Look at different metrics that might be more informative - for instance, precision and recall
3. Look at redundant variables - you learned this in scikit-learn, but EE has options for this too if you want to search for it.

It's up to you how to approach this, but we recommend to document what worked and what didn't work so that you have an easier time reporting your results.

In [ ]:
# Train the model
classifier = ee.Classifier.smileRandomForest(numberOfTrees=100, minLeafPopulation=2, maxNodes=50)
trained_classifier = classifier.train(features=train_data, classProperty=label_col, inputProperties=img_bands)

# Apply the classifier to the validation data
classified_val = val_data.classify(trained_classifier)

Now, let's analyze your results.

In [ ]:
# Calculate metrics
confusion_matrix = classified_val.errorMatrix(label_col, 'classification')
val_accuracy = confusion_matrix.accuracy()
kappa = confusion_matrix.kappa()

# Package all metrics into a single dictionary
metrics = ee.Dictionary({
    'Accuracy': val_accuracy,
    'Kappa': kappa
})

# Retrieve all metrics in one call
metrics_info = metrics.getInfo()

# Print all metrics
print('Validation Metrics:')
print(f"Accuracy: {metrics_info['Accuracy']}")
print(f"Kappa: {metrics_info['Kappa']}")

**Q: Describe the changes you made to the 'standard' approach.**
1. **Why did you make these changes**
2. **which impact did each change have on your analysis?**
3. **Which things did you try that did not work out?**

### Run on test set

Now, we load recent test images to classify recent urban extent. We take a median image over August 2024.

In [ ]:
# Load Sentinel-2 Image Collection
test_sentinel2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED") \
             .filterBounds(country_ROI.geometry()) \
             .filterDate('2024-08-01', '2024-08-31') \
             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))

# Create a median composite to reduce cloud cover
test_mosaic_image = sentinel2.median().clip(country_ROI.geometry())

# Add variables
test_mosaic_image = make_s2_variables(test_mosaic_image)

# Add to map object
map.addLayer(test_mosaic_image, vis_params, "test-Sentinel-2-2024")

Let's visualize the results calculated over the entire test image

In [ ]:
# Classify the test image
classified_image = test_mosaic_image.classify(trained_classifier)

# Define the color mapping dictionary
clc_colors = {
    0: '#FFFFFF', # Rural
    1: '#000000', # Urban
}

# Convert string labels to numeric codes
def classify_to_numeric(image):
    # Create a dictionary that maps string labels to numeric values
    label_to_numeric = {label: index for index, label in enumerate(clc_colors.keys())}

    # Convert string label to numeric value
    return image.remap(
        list(label_to_numeric.keys()),
        list(label_to_numeric.values())
    )

# Convert the classified image
numeric_classified_image = classify_to_numeric(classified_image)

# Generate a palette for visualization
palette = [clc_colors[label] for label in clc_colors.keys()]

# Add the numeric classified image to the map
map.addLayer(numeric_classified_image, {'palette': palette, 'min': 0, 'max': len(clc_colors) - 1}, 'Classified Image')
map

There are no metrics to calculate here - after all, we're trying to update our reference data. Instead, visually evaluate if you're happy with the results. When you're happy with the results, proceed to the next block to export the data from Earth Engine into our local environment.

(**hint:** don't go for perfect, this is very hard to achieve in the current setting)

**Q: Describe your map and your results in general.**
1. **Where do you still see errors? Are these systemic?**
2. **How could you further improve your product to clean these up?** **bold text**
3. **Was there anything you wanted to try, but were unable to?**

### Export resulting image

In [ ]:
geemap.ee_export_image(classified_image, filename='urban_rural_raster.tif', scale=1000, file_per_band=False)
urban_raster = xr.open_dataset('urban_rural_raster.tif', engine='rasterio')

### 3. Overlay urban and rural classification with population and Healthcare information

First, open the urban_rural dataset you just downloaded.

In [ ]:
urban_rural = xr.open_dataarray('XX')

In this step, we overlay the resulting urban/rural raster image from the previous section onto our population data point. a binary categorical variable, unban_rural, will be added to each data point, where 1 represents urban and 0 represents rural.

In [ ]:
values = urban_rural.sel(
                {
                    urban_rural.rio.x_dim: xr.DataArray(df_worldpop_.geometry.x),
                    urban_rural.rio.y_dim: xr.DataArray(df_worldpop_.geometry.y),
                },
                method="nearest",
            ).values[0]

df_worldpop_["urban_rural"] = ### add the values
df_worldpop_["urban_rural"] = ### fill the not a numbers with zeros.

lets check the resulted calssified population, as mentioned 1 means urban and 0 rural.

In [ ]:
df_worldpop_

## 4. Clustering population data points and assigning them to their nearest hospital.

In this section, we first add a local_ID to all hospitals to make sure each one has a unique id to be refered to later. then we create clusters of population. In this regard, we use K-means clustering algorithm with k equal to the number of hospitals. then calculate the sum of rural population each cluster as well as the urban population.  then we calcualte the euclidean distance of each center of cluster to all hospitals and assessing the hospital that has the minimum distance to them.

In [ ]:
len(HealthCenters_centroids)

In [ ]:
### 1st step: Add Local_ID to the HealthCenters_centroids GeoDataFrame
HealthCenters_centroids['Local_ID'] = range(1, len(HealthCenters_centroids) + 1)

# 2nd step: Ensure both GeoDataFrames are in the same CRS (EPSG:4326)
HealthCenters_centroids = HealthCenters_centroids.to_crs(

# Convert geometries to a list of coordinates (for KMeans)
pop_coords = np.array([(geom.x, geom.y) for geom in df_worldpop_['geometry']])
pop_band_data = df_worldpop_['band_data'].values

# Extract the hospital coordinates
hospital_coords = np.array([(geom.x, geom.y) for geom in HealthCenters_centroids['geometry']])
hospital_local_ids = HealthCenters_centroids['Local_ID'].values
hospital_geometries = HealthCenters_centroids['geometry'].values

### 3rd step: K-Means
# Ensure that the number of clusters is equal to the number of hospital coordinates
n_clusters =

kmeans = KMeans(n_clusters=n_clusters, random_state=random_seed, init=hospital_coords, n_init=1)  # using hospital locations as initial centers

df_worldpop_['cluster'] = kmeans.fit_predict(pop_coords)

# Get cluster centers (latitude & longitude)
cluster_centers = kmeans.cluster_centers_

### 4th step: calculate the sum of urban and rural populations in each cluster
# group by cluster and urban/rural status then sum up the populations

# rural population (urban_rural = 0)
rural_population_by_cluster = df_worldpop_[df_worldpop_['urban_rural'] == 0.0].groupby('cluster')['band_data'].sum().reindex(range(n_clusters), fill_value=0)

# urban population (urban_rural = 1)
urban_population_by_cluster = df_worldpop_[df_worldpop_['urban_rural'] == 1.0].groupby('cluster')['band_data'].sum().reindex(range(n_clusters), fill_value=0)

# to create a new DataFrame for clusters with the total urban and rural population
clusters_df = pd.DataFrame({
    'cluster': range(n_clusters),
    'geometry': [Point(x, y) for x, y in cluster_centers],
    'urban_population': urban_population_by_cluster.values,
    'rural_population': rural_population_by_cluster.values
})

### 5th step: assign the nearest hospital to the related cluster
# distances between each cluster center and each health facility center
distances = cdist(cluster_centers, hospital_coords, metric='euclidean')

# the index of the nearest hospital for each cluster
nearest_hospital_idx = distances.argmin(axis=1)

# assign the nearest hospital Local_ID and geometry to each cluster center
clusters_df['nearest_hospital_local_id'] = [hospital_local_ids[idx] for idx in nearest_hospital_idx]
clusters_df['nearest_hospital_geometry'] = [hospital_geometries[idx] for idx in nearest_hospital_idx]

### 6th step: convert to GeoDataFrame and set CRS
clusters_gdf = gpd.GeoDataFrame(clusters_df, geometry='geometry')
clusters_gdf.set_crs(epsg=4326, inplace=True)

# review the content of the resulting clusters dataframe
clusters_gdf


## 5. Explore and evaluate baseline results

**Q: Plot cluster centers and the location of hospitals**

### Identify and plot population per healthcare facility

Here, we calculate the urban and rural demand for services for each hospital by summing the populations in each cluster assigned to the hospitals. We also calculate the total demand for each hospital.

In [ ]:
# calculating the total rural and urban demand (population) of each hospital
hospital_population = clusters_gdf.groupby('nearest_hospital_local_id')[['urban_population', 'rural_population']].sum().reset_index()

# now we merge the population data back to the HealthCenters_centroids GeoDataFrame
hospital_population_merged = HealthCenters_centroids.merge(hospital_population, left_on='Local_ID', right_on='nearest_hospital_local_id', how='left')

# also we sum up rural and urban demand to get the total demand of each hospital as well
hospital_population_merged['total_population'] = hospital_population_merged['urban_population'] + hospital_population_merged['rural_population']

# display the demand of hospitals
hospital_population_merged[['Local_ID', 'urban_population', 'rural_population', 'total_population']].sort_values('total_population', ascending=False).dropna()


**Q: Plot the Hospital locations and their total demand**

Now we check if there is any hospital that has not been assigned to any population cluster.

In [ ]:
assigned_hospitals = clusters_gdf['nearest_hospital_local_id'].unique()

unassigned_hospitals = HealthCenters_centroids[~HealthCenters_centroids['Local_ID'].isin(assigned_hospitals)]

if unassigned_hospitals.empty:
    print("All hospitals are assigned to at least one cluster.")
else:
    print("following hospitals are not assigned to any cluster:")
    print(unassigned_hospitals[['Local_ID', 'geometry']])

## 6. Natural hazard disruption

### Download flood data
The flood data we will extract from a repository maintained by the World Resources Institute. We will download river flood hazard maps from their [Flood Data Collection](https://wri-projects.s3.amazonaws.com/AqueductFloodTool/download/v2/index.html).

Here we do not need to use an API and we also do not need to register ourselves, so we can download any of the files directly. In case you do not have any flood impacts. You could select a more extreme future scenario. But let's first download the 1/1000 river flood event under historic conditions.

### Overlay flood data with Hospitals

In [ ]:
flood_map_path = "XX"

In [ ]:
flood_map = xr.open_dataset(flood_map_path, engine="rasterio")
flood_map

As you can see, this is a very large dataset again. Let's make our life a little bit more relaxed by clipping the map to our country of interest. We use the **rio.clip_box** function to do so.

In [ ]:
min_lon =  country_geom.bounds.minx.
min_lat =  #complete function
max_lon =  #complete function
max_lat =  #complete function

flood_map_area = flood_map.rio.clip_box(minx=min_lon,miny=min_lat,maxx=max_lon,maxy=max_lat) #complete function

And plot the data to explore how it looks.

And make sure you convert it to a coordinate system in meters, to do a correct overlay in the following step.

### Overlay flood data with healthcare facilities

The next step is to overlay the flood data with the healthcare facilities. To do so, we will estimate the potential damage that can occur to the various hospitals in the country of interest.



In [ ]:
def _get_damage_per_object(asset, curves, cell_area_m2):
    """
    Calculate damage for a given asset based on hazard information.
    Arguments:
        *asset*: Tuple containing information about the asset. It includes:
            - Index or identifier of the asset (asset[0]).
            - Asset-specific information, including hazard points (asset[1]['hazard_point']).
        *maxdam_dict*: Maximum damage value.
    Returns:
        *tuple*: A tuple containing the asset index or identifier and the calculated damage.
    """

    if asset.geometry.geom_type in ("Polygon", "MultiPolygon"):
        coverage = asset["coverage"] * cell_area_m2
    elif asset.geometry.geom_type in ("LineString", "MultiLineString"):
        coverage = asset["coverage"]
    elif asset.geometry.geom_type in ("Point"):
        coverage = 1
    else:
        raise ValueError(f"Geometry type {asset.geometry.geom_type} not supported")

    return (
        np.sum(
            np.interp(
                asset["values"], curves.index, curves[asset["amenity"]].values
            )
            * coverage
        )
        * asset["maximum_damage"]
    )

We predefine some simple maximum damages and vulnerability curves. Feel free to adjust the damages and curves if you found better information.

In [ ]:
maxdam = {"hospital":2000,
        "clinic":1500,
}

curves = np.array(
            [[0,0],
            [50,0.2],
            [100,0.4],
            [150,0.6],
            [200,0.8],
            [250,1]])

curves = np.concatenate((curves,
                            np.transpose(np.array([curves[:,1]]*(len(maxdam)-1)))),
                           axis=1)

curves = pd.DataFrame(curves)
curves.columns = ['depth']+list(maxdam.keys())
curves.set_index('depth',inplace=True)

And now we apply **exact_extract** to assess the overlay between the flood data and the Healthcare facilities.

In [ ]:
values_and_coverage_per_object = exact_extract(
            flood_map,
            HealthCenters.to_crs(3857),
            ["coverage", "values"],
            output="pandas",
        )

Now we merge the **HealthCenters** data and the **values_and_coverage_per_object data**.

In [ ]:
HealthCenters = HealthCenters.merge( XXX ,left_index=True,right_index=True)

And assign a maximum damage per subtype.

In [ ]:
HealthCenters['maximum_damage'] = HealthCenters.amenity.apply(lambda x: maxdam[x])

And now assess the potential damage. The cell_area should correspond to the cell_size of the flood map.

In [ ]:
HealthCenters['damage'] = HealthCenters.apply(
        lambda _object: _get_damage_per_object(_object, curves, cell_area_m2=1000*1000),
        axis=1,
    )

And lets explore the results. Perhaps plot them again?

## 7. Your Final Task

As you saw, due to a flood with a 1000-year return period, some hospitals may be out of service. Therefore, we need to estimate the post-flood urban/rural demand for services from the hospitals that remain operational.

### Your task here will be:
- Create new clusters of populations and assign them to the remaining hospitals, then determine the post-disaster demand for these hospitals.
- Calculate the urban, rural and total demand (population in need of services) for each hospital.
- Plot the remaining hospitals vs their total population in need of service.
- Let us know how many hospitals were affected by the flood and the total number of rural residents who need to find an alternative hospital.